In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller

In [2]:
#Import function for cleaning, merged, and import data from cleaning_data.py and Import_and_Cleaning.py
sys.path.append('../src/Import_and_Cleaning')
from import_and_merge_data import load_and_merge_all_data

df_main = load_and_merge_all_data()

In [3]:
# --- Data Transformation and Preparation ---
df_analysis = df_main.copy()
# ---  Log‐transform GDP per capita ---
df_analysis['log_GDP_percapita'] = np.log(df_analysis['GDP_percapita'])

#### Multicolliniearity

In [4]:
# Prepare the design‐matrix for VIF
features = [
    'Dependency_Ratio_Old',
    'Dependency_Ratio_Young',
    'log_GDP_percapita',
    'trade_gdp'
]

# Subset, drop any NaN/∞, force float
X = (
    df_analysis[features]
      .replace([np.inf, -np.inf], np.nan)
      .dropna()
      .astype(float)
)

In [5]:
# Build VIF table
vif = pd.DataFrame({
    'variable': features,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})
print(vif)

                 variable        VIF
0    Dependency_Ratio_Old  10.926990
1  Dependency_Ratio_Young   7.580817
2       log_GDP_percapita  32.781942
3               trade_gdp   3.559165


**Multicollinearity diagnostics**  
- Variance inflation factors (VIF):  
  - Old-age dependency ratio: 10.93  
  - Youth dependency ratio: 7.58  
  - log (GDP per capita): 32.78  
  - Trade /GDP: 3.56  
- The elevated VIFs for the two dependency ratios (and especially for log GDPpc) reflect structural, demographic correlations rather than a sampling anomaly.  
- **Acknowledge & interpret with caution**: we retain both dependency ratios in the main TWFE model and discuss their coefficients as reflecting shifts in overall population structure, not “pure” ceteris-paribus effects.  
- As robustness checks, we will also estimate two simpler specifications—one with only the old-age ratio and one with only the youth ratio—to show how the coefficients move when the other ratio is omitted.  


In [6]:
# Prepare the design‐matrix for VIF
features = [
    'Dependency_Ratio_Old',
    'log_GDP_percapita',
    'trade_gdp'
]

# Subset, drop any NaN/∞, force float
X = (
    df_analysis[features]
      .replace([np.inf, -np.inf], np.nan)
      .dropna()
      .astype(float)
)

In [7]:
# Build VIF table
vif = pd.DataFrame({
    'variable': features,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})
print(vif)

               variable       VIF
0  Dependency_Ratio_Old  6.116927
1     log_GDP_percapita  9.741903
2             trade_gdp  3.329912


#### Stationary Check

In [8]:
# --- Panel Stationarity Check: ADF in Levels vs. First Differences ---
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd

# 1) Variables to test
test_vars = [
    'Health_Expenditure',
    'Dependency_Ratio_Old',
    'Dependency_Ratio_Young',
    'log_GDP_percapita',
    'trade_gdp'
]

# 2) Strip non‐numeric characters & coerce to float
for var in test_vars:
    # remove %, commas, any other non‐digit/dot/minus
    df_analysis[var] = (
        df_analysis[var]
          .astype(str)
          .str.replace('%',    '', regex=False)
          .str.replace(',',    '', regex=False)
          .str.replace(r'[^0-9\.\-]', '', regex=True)
    )
    df_analysis[var] = pd.to_numeric(df_analysis[var], errors='coerce')

# 4) Build a panel DataFrame
df_panel = df_analysis.set_index(['ISO3', 'Year']).sort_index()

# 5) Run per‐country ADF in levels & first differences
results = []
for var in test_vars:
    p_lv = []
    p_df = []
    for iso, grp in df_panel.groupby(level=0):
        series = grp[var].dropna()
        if len(series) < 10:
            continue
        # ADF on levels
        stat, pval, *_ = adfuller(series, autolag='AIC')
        p_lv.append(pval)
        # ADF on first differences
        stat, pval, *_ = adfuller(series.diff().dropna(), autolag='AIC')
        p_df.append(pval)

    results.append({
        'Variable':         var,
        'Avg. p-level':     np.mean(p_lv),
        'Avg. p-Δ':         np.mean(p_df),
        'Frac stationary Δ': np.mean([p < 0.05 for p in p_df])
    })

# 6) Display results
pd.DataFrame(results).round(3)


,Variable,Avg. p-level,Avg. p-Δ,Frac stationary Δ
0,Health_Expenditure,0.519,0.235,0.593
1,Dependency_Ratio_Old,0.873,0.565,0.093
2,Dependency_Ratio_Young,0.296,0.320,0.389
3,log_GDP_percapita,0.358,0.246,0.481
4,trade_gdp,0.457,0.372,0.442


**Stationarity diagnostics**  
Our per‐country ADF tests show that all five series—health expenditure, the old‐age and youth dependency ratios, log GDP per capita, and trade/GDP—fail to reject the null of a unit root in levels (average p-values > 0.05), confirming their non-stationary behavior. First differencing lowers the p-values. While not every series becomes strictly stationary after differencing, the Δ transformation substantially reduces persistence and mitigates the risk of spurious regression.

Accordingly, we proceed with a first‐difference specification in our TWFE models—recognizing in our write-up that some variables, particularly the old‐age dependency ratio, remain highly persistent even in Δ form.  


In [9]:
# --- Step 3: First‐difference for stationarity ---
# 3a) sort by country and year, work on a copy
df_diff = df_analysis.sort_values(['ISO3', 'Year']).copy()

# 3b) variables to difference (including the dependent)
to_diff = [
    'Dependency_Ratio_Old',
    'Dependency_Ratio_Young',
    'log_GDP_percapita',
    'trade_gdp',
    'Health_Expenditure'
]

# 3c) compute year‐on‐year change for each series
for v in to_diff:
    df_diff[f'd_{v}'] = df_diff.groupby('ISO3')[v].diff()

# 3d) drop the first‐difference NaNs
df_diff = df_diff.dropna(subset=[f'd_{v}' for v in to_diff])

# 3e) inspect the new Δ-variables
df_diff[['ISO3','Year'] + [f'd_{v}' for v in to_diff]].head()


,ISO3,Year,d_Dependency_Ratio_Old,d_Dependency_Ratio_Young,d_log_GDP_percapita,d_trade_gdp,d_Health_Expenditure
1,ARG,2001,0.066719,-0.464572,-0.067087,-0.770189,0.151788
2,ARG,2002,0.062658,-0.494729,-1.022130,19.900469,-0.978487
3,ARG,2003,0.037586,-0.531833,0.256320,-1.107976,-0.498795
4,ARG,2004,0.031726,-0.556617,0.244931,0.047898,0.334166
5,ARG,2005,0.058268,-0.560617,0.177838,-0.141375,0.382106


## OLS

In [10]:
import statsmodels.formula.api as smf

# 0) Subset your sample to Year ≤ 2022
df_model = df_analysis[df_analysis['Year'] <= 2022].copy()

# 1) Pooled OLS
formula_ols = (
    'Health_Expenditure ~ '
    'Dependency_Ratio_Old +  '
    'log_GDP_percapita + trade_gdp'
)

pooled_ols = smf.ols(formula_ols, data=df_model).fit()
print("=== Model 1: Pooled OLS ===")
print(pooled_ols.summary())


=== Model 1: Pooled OLS ===
                            OLS Regression Results                            
Dep. Variable:     Health_Expenditure   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     366.1
Date:                Sun, 22 Jun 2025   Prob (F-statistic):          5.19e-168
Time:                        10:18:34   Log-Likelihood:                -2470.2
No. Observations:                1184   AIC:                             4948.
Df Residuals:                    1180   BIC:                             4969.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Inte

## Country FE

In [11]:
# 2) Country FE
formula_fe_country = (
    'Health_Expenditure ~ '
    'Dependency_Ratio_Old + '
    'log_GDP_percapita + trade_gdp + '
    'C(ISO3)'
)

fe_country = smf.ols(formula_fe_country, data=df_model).fit()
print("\n=== Model 2: Country FE ===")
print(fe_country.summary())



=== Model 2: Country FE ===
                            OLS Regression Results                            
Dep. Variable:     Health_Expenditure   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     224.8
Date:                Sun, 22 Jun 2025   Prob (F-statistic):               0.00
Time:                        10:18:34   Log-Likelihood:                -1400.9
No. Observations:                1184   AIC:                             2912.
Df Residuals:                    1129   BIC:                             3191.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Int

## Covid and Country FE

In [12]:
import statsmodels.formula.api as smf

# 1) Build your base sample (Year ≤ 2022) and Covid dummies
df_model = df_analysis[df_analysis['Year'] <= 2022].copy()
df_model['D_2020'] = (df_model['Year'] == 2020).astype(int)
df_model['D_2021'] = (df_model['Year'] == 2021).astype(int)

# 2) Drop rows missing any regressor or FE key
needed = [
    'Health_Expenditure',
    'Dependency_Ratio_Old',
    'log_GDP_percapita',
    'trade_gdp',
    'ISO3',
    'D_2020',
    'D_2021'
]
df_model2 = df_model.dropna(subset=needed)

# 3) TWFE with only Country FE + two year dummies, default (non‐clustered) SE
formula_covid = (
    'Health_Expenditure ~ '
    'Dependency_Ratio_Old + '
    'log_GDP_percapita + trade_gdp + '
    'C(ISO3) + D_2020 + D_2021'
)

twfe_covid_plain = smf.ols(formula_covid, data=df_model2).fit()

print("=== TWFE controlling only 2020 & 2021, non‐clustered SE ===")
print(twfe_covid_plain.summary())


=== TWFE controlling only 2020 & 2021, non‐clustered SE ===
                            OLS Regression Results                            
Dep. Variable:     Health_Expenditure   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     221.2
Date:                Sun, 22 Jun 2025   Prob (F-statistic):               0.00
Time:                        10:18:34   Log-Likelihood:                -1389.1
No. Observations:                1184   AIC:                             2892.
Df Residuals:                    1127   BIC:                             3182.
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

clustered Std Error

In [13]:
import statsmodels.formula.api as smf

# 1) Build your df_model as before (Year ≤ 2022 + dummies)
df_model = df_analysis[df_analysis['Year'] <= 2022].copy()
df_model['D_2020'] = (df_model['Year'] == 2020).astype(int)
df_model['D_2021'] = (df_model['Year'] == 2021).astype(int)

# 2) List every variable in your formula
needed = [
    'Health_Expenditure',
    'Dependency_Ratio_Old',
    'log_GDP_percapita',
    'trade_gdp',
    'ISO3',
    'D_2020',
    'D_2021'
]

# 3) Drop any row missing *any* of those
df_model2 = df_model.dropna(subset=needed)

# 4) Re-run the regression, now passing the *trimmed* ISO3 vector
formula_covid = (
    'Health_Expenditure ~ '
    'Dependency_Ratio_Old + '
    'log_GDP_percapita + trade_gdp +'
    'C(ISO3) + D_2020 + D_2021'
)

twfe_covid_cluster = smf.ols(formula_covid, data=df_model2).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_model2['ISO3'].values}
)

print(twfe_covid_cluster.summary())


                            OLS Regression Results                            
Dep. Variable:     Health_Expenditure   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     20.04
Date:                Sun, 22 Jun 2025   Prob (F-statistic):           5.32e-11
Time:                        10:18:34   Log-Likelihood:                -1389.1
No. Observations:                1184   AIC:                             2892.
Df Residuals:                    1127   BIC:                             3182.
Df Model:                          56                                         
Covariance Type:              cluster                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                2.9475 

/Users/gilang/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 56, but rank is 5
  warnings.warn('covariance of constraints does not have full '


#### Lag Model

In [14]:
import statsmodels.formula.api as smf

# 1) Build & trim your base DataFrame
df_model = df_analysis[df_analysis['Year'] <= 2022].copy()
df_model['D_2020'] = (df_model['Year'] == 2020).astype(int)
df_model['D_2021'] = (df_model['Year'] == 2021).astype(int)

# 2) Create a 1‐year lag of Dependency_Ratio_Old
df_model['DepOld_lag1'] = (
    df_model.groupby('ISO3')['Dependency_Ratio_Old']
            .shift(1)
)

# 3) Drop any rows missing one of the RHS vars
needed = [
    'Health_Expenditure',
    'Dependency_Ratio_Old',
    'DepOld_lag1',
    'log_GDP_percapita',
    'trade_gdp',
    'ISO3',
    'D_2020',
    'D_2021'
]
df2 = df_model.dropna(subset=needed)

# 4) Estimate TWFE + COVID dummies + 1‐year lag
formula_lag1 = (
    'Health_Expenditure ~ '
    'Dependency_Ratio_Old + DepOld_lag1 + '
    'log_GDP_percapita + trade_gdp +'
    'C(ISO3) + D_2020 + D_2021'
)

twfe_lag1 = smf.ols(formula_lag1, data=df2).fit(
    cov_type='cluster',
    cov_kwds={'groups': df2['ISO3'].values}
)
print(twfe_lag1.summary())


                            OLS Regression Results                            
Dep. Variable:     Health_Expenditure   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     56.96
Date:                Sun, 22 Jun 2025   Prob (F-statistic):           6.89e-21
Time:                        10:18:34   Log-Likelihood:                -1283.0
No. Observations:                1133   AIC:                             2682.
Df Residuals:                    1075   BIC:                             2974.
Df Model:                          57                                         
Covariance Type:              cluster                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                3.8867 

/Users/gilang/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 57, but rank is 6
  warnings.warn('covariance of constraints does not have full '


### First Difference

In [15]:
# --- Model 5: First‐Difference regression ---
# 5a) Subset the differenced data to Year ≤ 2022
df_diff_model = df_diff[df_diff['Year'] <= 2022].copy()

# 5b) Specify the formula on your Δ-variables
formula_diff = (
    'd_Health_Expenditure ~ '
    'd_Dependency_Ratio_Old + '
    'd_log_GDP_percapita + d_trade_gdp'
)

# 5c) Fit the OLS on first-differences
fd_clust = smf.ols(formula_diff, data=df_diff_model).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_diff_model['ISO3'].values}
)

print("\n=== Model 5: Δ-model with country‐clustered SE ===")
print(fd_clust.summary())


=== Model 5: Δ-model with country‐clustered SE ===
                             OLS Regression Results                             
Dep. Variable:     d_Health_Expenditure   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                  0.036
Method:                   Least Squares   F-statistic:                     7.537
Date:                  Sun, 22 Jun 2025   Prob (F-statistic):           0.000288
Time:                          10:18:34   Log-Likelihood:                -680.34
No. Observations:                  1132   AIC:                             1369.
Df Residuals:                      1128   BIC:                             1389.
Df Model:                             3                                         
Covariance Type:                cluster                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------

In [16]:
# --- 1) Pick your differenced model DataFrame (Year ≤ 2022) ---
df_diff_model = df_diff[df_diff['Year'] <= 2022].copy()

# --- 2) Create a 1-year lag of the Δ old‐age ratio ---
df_diff_model['d_DepOld_lag1'] = (
    df_diff_model
      .groupby('ISO3')['d_Dependency_Ratio_Old']
      .shift(1)
)

# --- 3) Drop any rows with missing lag (or any other NaN) ---
needed = [
    'd_Health_Expenditure',
    'd_Dependency_Ratio_Old',
    'd_DepOld_lag1',
    'd_Dependency_Ratio_Young',
    'd_log_GDP_percapita',
    'd_trade_gdp'
]
df_ld = df_diff_model.dropna(subset=needed)

# --- 4) Specify & run the “lagged Δ‐model” ---
import statsmodels.formula.api as smf

formula_lag = (
    'd_Health_Expenditure ~ '
    'd_Dependency_Ratio_Old + d_DepOld_lag1 + '
    'd_Dependency_Ratio_Young + '
    'd_log_GDP_percapita + d_trade_gdp'
)

ld_ols = smf.ols(formula_lag, data=df_ld).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_ld['ISO3'].values}
)

print(ld_ols.summary())


                             OLS Regression Results                             
Dep. Variable:     d_Health_Expenditure   R-squared:                       0.051
Model:                              OLS   Adj. R-squared:                  0.047
Method:                   Least Squares   F-statistic:                     7.978
Date:                  Sun, 22 Jun 2025   Prob (F-statistic):           1.30e-05
Time:                          10:18:34   Log-Likelihood:                -655.38
No. Observations:                  1080   AIC:                             1323.
Df Residuals:                      1074   BIC:                             1353.
Df Model:                             5                                         
Covariance Type:                cluster                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Inte

### Presentation

In [17]:
from statsmodels.iolib.summary2 import summary_col

In [18]:
# 1) collect your fitted models
models = [
    pooled_ols,
    fe_country,
    twfe_covid_plain,
    twfe_covid_cluster,
    twfe_lag1
]

# 2) the column headers you want
model_names = [
    "Pooled OLS",
    "Country FE",
    "Country & Covid FE",
    "Country & Covid FE (clustered)",
    "Lag"
]

# 3) pick out exactly the regressors you care about (in the order you want them)
regressor_order = [
    "Dependency_Ratio_Old",
    "DepOld_lag1",   # only present in the last model
    "log_GDP_percapita",
    "trade_gdp"
]

# 4) build your bottom‐of‐table stats
info_dict = {
    "Observations": lambda m: f"{int(m.nobs):,}",
    "R-squared":     lambda m: f"{m.rsquared:.3f}",
}

# 5) create the summary
summary = summary_col(
    results=models,
    model_names=model_names,
    regressor_order=regressor_order,
    info_dict=info_dict,
    stars=True,            # adds *,**,***
    drop_omitted=True,
    float_format="%0.3f"
)

# 6) pull out the DataFrame and rename your rows/variables for prettiness
df = summary.tables[0]

df = df.rename(index={
    "Dependency_Ratio_Old":       "Dependency Ratio (Old)",
    "DepOld_lag1":                "Lagged Dependency Ratio (Old)",
    "log_GDP_percapita":          "Log GDP per capita",
    "trade_gdp":                  "Trade-to-GDP"
})

# 7) append your FE indicators
df.loc["Country FE"] = ["No","Yes","Yes","Yes","Yes"]
df.loc["Covid FE"]   = ["No","No","Yes","Yes","Yes"]

# 8) display
print(df)


                              Pooled OLS Country FE Country & Covid FE  \
Dependency Ratio (Old)          0.114***   0.182***           0.162***   
                                 (0.010)    (0.011)            (0.012)   
Lagged Dependency Ratio (Old)                                            
                                                                         
Log GDP per capita              0.781***   0.394***           0.379***   
                                 (0.060)    (0.055)            (0.054)   
Trade-to-GDP                   -0.010***   -0.003**             -0.002   
                                 (0.001)    (0.002)            (0.002)   
R-squared                          0.482      0.915              0.917   
R-squared Adj.                     0.481      0.911              0.912   
Observations                       1,184      1,184              1,184   
R-squared                          0.482      0.915              0.917   
Country FE                            

In [21]:
# summary is the object you got from summary_col(...)
latex_code = summary.as_latex()

with open("regression_table.tex", "w") as f:
    f.write(latex_code)


In [ ]:
# Markdown for Jupyter or GitHub
print(df.to_markdown())

|                               | Pooled OLS   | Country FE   | Country & Covid FE   | Country & Covid FE (clustered)   | Lag      |
|:------------------------------|:-------------|:-------------|:---------------------|:---------------------------------|:---------|
| Dependency Ratio (Old)        | 0.114***     | 0.182***     | 0.162***             | 0.162***                         | 0.558*** |
|                               | (0.010)      | (0.011)      | (0.012)              | (0.026)                          | (0.216)  |
| Lagged Dependency Ratio (Old) |              |              |                      |                                  | -0.416*  |
|                               |              |              |                      |                                  | (0.221)  |
| Log GDP per capita            | 0.781***     | 0.394***     | 0.379***             | 0.379**                          | 0.310**  |
|                               | (0.060)      | (0.055)      | (0.05

In [24]:
# Excel for easy styling in Excel/Sheets
df.to_excel("regression_table.xlsx", sheet_name="Results", index=True)